# Choose baseline

In [3]:
from utils import load_data

split = 'valid'

data_path = '/ivi/ilps/personal/svakule/spoken_qa'
dataset_versions = ['WD18_entities_original',
                    'WD18_entities_wav2vec2-base-960h',
                    'WD18_relations_original',
                    'WD18_relations_wav2vec2-base-960h']

In [8]:
from beir.retrieval.search.dense import DenseRetrievalExactSearch as DRES
from beir.retrieval.evaluation import EvaluateRetrieval
from beir.retrieval import models

baselines = [["msmarco-distilbert-base-v2", ''],
             ["msmarco-distilbert-base-v3", 'cos_sim'],
             ["msmarco-roberta-base-v3", 'cos_sim'],
             ["msmarco-distilbert-base-dot-prod-v3", 'dot'],
             ["msmarco-distilbert-base-tas-b", 'dot']]

similarities = ['cos_sim', 'dot']


for model_save_path, original_similarity in baselines[1:2]:
    # load model
    model = DRES(models.SentenceBERT(model_save_path))
    
    # wiggle similarity function at inference time
    for similarity in similarities:
        print(model_save_path, original_similarity, similarity)
        retriever = EvaluateRetrieval(model, score_function=similarity)

        # iterate over all dataset versions
        for dataset_version in dataset_versions[:]:
            corpus, queries, qrels = load_data(split, dataset_version, data_path)

            results = retriever.retrieve(corpus, queries)
            ndcg, _map, recall, precision = retriever.evaluate(qrels, results, retriever.k_values)
            print('%.3f\t'%precision['P@1'])
    #         print('%.3f\t'*5 % (recall['Recall@1'], recall['Recall@3'],
    #                             recall['Recall@5'], recall['Recall@10'],
    #                             recall['Recall@100']))
            break

msmarco-distilbert-base-v3 cos_sim cos_sim
WD18 entities original valid


Batches:   0%|          | 0/26 [00:00<?, ?it/s]

Batches:   0%|          | 0/223 [00:00<?, ?it/s]

0.915	
msmarco-distilbert-base-v3 cos_sim dot
WD18 entities original valid


Batches:   0%|          | 0/26 [00:00<?, ?it/s]

Batches:   0%|          | 0/223 [00:00<?, ?it/s]

0.916	


# Evaluate

In [3]:
import pathlib, os
from beir.datasets.data_loader import GenericDataLoader

data_path = '/ivi/ilps/personal/svakule/spoken_qa'
split = 'valid'
dataset_versions = ['WD18_entities_original',
                    'WD18_entities_wav2vec2-base-960h',
                    'WD18_relations_original',
                    'WD18_relations_wav2vec2-base-960h']

In [6]:
from beir.retrieval.search.dense import DenseRetrievalExactSearch as DRES
from beir.retrieval.evaluation import EvaluateRetrieval
from beir.retrieval import models

from utils import load_data

model_path = '/ivi/ilps/personal/svakule/spoken_qa/models/'
model_names = [["msmarco-distilbert-base-v3", 'cos_sim'],
               ["msmarco-distilbert-base-tas-b", 'dot'],
               [model_path+"msmarco-distilbert-base-v3-cos_sim-WD18_entities_original", 'cos_sim'],
               [model_path+"msmarco-distilbert-base-tas-b-dot-WD18_entities_original", 'dot'],
               [model_path+"msmarco-distilbert-base-tas-b-cos_sim-WD18_entities_original", 'cos_sim'],
               [model_path+"msmarco-distilbert-base-tas-b-dot-WD18_relations_original", 'dot'],
               [model_path+"msmarco-distilbert-base-tas-b-cos_sim-WD18_relations_original", 'cos_sim']]

# similarities = ['cos_sim', 'dot']


# iterate over models
for model_name, similarity in model_names[:]:
    # load model
    model = DRES(models.SentenceBERT(model_name))
    
    # wiggle similarity function at inference time
#     for similarity in similarities:
    print(model_name, similarity)
    retriever = EvaluateRetrieval(model, score_function=similarity)

    # iterate over datasets
    for dataset_version in dataset_versions[:]:
        corpus, queries, qrels = load_data(split, dataset_version, data_path)

        results = retriever.retrieve(corpus, queries)
        ndcg, _map, recall, precision = retriever.evaluate(qrels, results, retriever.k_values)
        print('%.3f\t'*5 % (recall['Recall@1'], recall['Recall@3'],
                            recall['Recall@5'], recall['Recall@10'],
                            recall['Recall@100']))
#         print('%.3f\t'%precision['P@1'])

msmarco-distilbert-base-v3 cos_sim
WD18 entities original valid


Batches:   0%|          | 0/26 [00:00<?, ?it/s]

Batches:   0%|          | 0/223 [00:00<?, ?it/s]

0.915	0.961	0.970	0.978	0.988	
WD18 entities wav2vec2-base-960h valid


Batches:   0%|          | 0/26 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/70 [00:00<?, ?it/s]

0.719	0.873	0.925	0.956	0.986	
WD18 relations wav2vec2-base-960h valid


Batches:   0%|          | 0/38 [00:00<?, ?it/s]

Batches:   0%|          | 0/70 [00:00<?, ?it/s]

0.547	0.678	0.729	0.772	0.842	
